# RNN Basiques avec TensorFlow

Ce notebook explore les concepts fondamentaux des Réseaux de Neurones Récurrents (RNN) en utilisant TensorFlow et Keras.

## Objectifs du notebook

1. Comprendre l'architecture des RNN
2. Implémenter un RNN simple avec TensorFlow/Keras
3. Explorer les différents types d'architectures RNN
4. Entraîner un RNN pour l'analyse de sentiment
5. Visualiser le fonctionnement interne d'un RNN

## 1. Installation et imports

In [1]:
# Installation des dépendances
!pip install tensorflow numpy pandas matplotlib seaborn scikit-learn

ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Configuration de l'affichage
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Vérifier la version de TensorFlow
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponible: {tf.config.list_physical_devices('GPU')}")

ModuleNotFoundError: No module named 'tensorflow'

## 2. Comprendre les RNN : Théorie et Intuition

In [ ]:
# Visualisation du concept de RNN
def visualize_rnn_concept():
    """
    Visualise le déroulement temporel d'un RNN.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # RNN replié
    ax1.set_title("RNN Replié", fontsize=16, fontweight='bold')
    
    # Dessiner la cellule RNN
    circle = plt.Circle((0.5, 0.5), 0.3, color='lightblue', ec='darkblue', linewidth=2)
    ax1.add_patch(circle)
    ax1.text(0.5, 0.5, 'RNN', ha='center', va='center', fontsize=14, fontweight='bold')
    
    # Flèches
    ax1.arrow(0.5, 0.9, 0, -0.08, head_width=0.05, head_length=0.02, fc='green', ec='green')
    ax1.text(0.5, 0.95, 'x_t', ha='center', fontsize=12, color='green')
    
    ax1.arrow(0.5, 0.18, 0, -0.08, head_width=0.05, head_length=0.02, fc='red', ec='red')
    ax1.text(0.5, 0.05, 'h_t', ha='center', fontsize=12, color='red')
    
    # Boucle récurrente
    ax1.annotate('', xy=(0.2, 0.5), xytext=(0.8, 0.5),
                arrowprops=dict(arrowstyle='<->', connectionstyle='arc3,rad=.5', 
                              color='purple', linewidth=2))
    ax1.text(0.5, 0.3, 'h_{t-1}', ha='center', fontsize=12, color='purple')
    
    ax1.set_xlim(0, 1)
    ax1.set_ylim(0, 1)
    ax1.axis('off')
    
    # RNN déroulé
    ax2.set_title("RNN Déroulé dans le Temps", fontsize=16, fontweight='bold')
    
    # Dessiner plusieurs cellules RNN
    positions = [0.2, 0.4, 0.6, 0.8]
    times = ['t-1', 't', 't+1', 't+2']
    
    for i, (pos, time) in enumerate(zip(positions, times)):
        # Cellule
        circle = plt.Circle((pos, 0.5), 0.08, color='lightblue', ec='darkblue', linewidth=2)
        ax2.add_patch(circle)
        ax2.text(pos, 0.5, 'RNN', ha='center', va='center', fontsize=10, fontweight='bold')
        
        # Input
        ax2.arrow(pos, 0.7, 0, -0.08, head_width=0.02, head_length=0.02, fc='green', ec='green')
        ax2.text(pos, 0.75, f'x_{time}', ha='center', fontsize=10, color='green')
        
        # Output
        ax2.arrow(pos, 0.4, 0, -0.08, head_width=0.02, head_length=0.02, fc='red', ec='red')
        ax2.text(pos, 0.25, f'h_{time}', ha='center', fontsize=10, color='red')
        
        # Connexions horizontales
        if i < len(positions) - 1:
            ax2.arrow(pos + 0.08, 0.5, 0.12, 0, head_width=0.03, head_length=0.02, 
                     fc='purple', ec='purple')
    
    ax2.set_xlim(0, 1)
    ax2.set_ylim(0, 1)
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()

visualize_rnn_concept()

## 3. Implémentation d'un RNN Simple

In [ ]:
# Implémentation d'un RNN simple avec Keras
def create_simple_rnn(input_size, hidden_size, output_size, sequence_length):
    """
    Crée un modèle RNN simple avec TensorFlow/Keras.
    
    Args:
        input_size: Dimension des features d'entrée
        hidden_size: Dimension de l'état caché
        output_size: Dimension de la sortie
        sequence_length: Longueur des séquences
    """
    model = models.Sequential([
        # Couche RNN
        layers.SimpleRNN(hidden_size, 
                        activation='tanh',
                        return_sequences=False,  # Ne retourner que la dernière sortie
                        input_shape=(sequence_length, input_size)),
        
        # Couche de sortie
        layers.Dense(output_size, activation='softmax')
    ])
    
    return model

# Créer un modèle exemple
model = create_simple_rnn(
    input_size=300,      # Dimension des embeddings
    hidden_size=128,     # Taille de l'état caché
    output_size=2,       # Classification binaire
    sequence_length=10   # Longueur des séquences
)

# Afficher l'architecture
model.summary()

In [ ]:
# Test avec des données synthétiques
batch_size = 32
sequence_length = 10
input_dim = 300

# Générer des données d'exemple
sample_input = np.random.randn(batch_size, sequence_length, input_dim)
print(f"Input shape: {sample_input.shape}")

# Forward pass
output = model(sample_input)
print(f"Output shape: {output.shape}")
print(f"\nPremières prédictions:\n{output[:5].numpy()}")

## 4. Différentes Architectures RNN

In [ ]:
# One-to-Many : Génération de séquence
def create_one_to_many_rnn(input_size, hidden_size, output_length, vocab_size):
    """
    RNN pour génération de séquence (ex: description d'image).
    """
    model = models.Sequential([
        # Input unique transformé en état initial
        layers.Dense(hidden_size, activation='relu', input_shape=(input_size,)),
        layers.RepeatVector(output_length),  # Répéter pour chaque timestep
        
        # RNN pour générer la séquence
        layers.SimpleRNN(hidden_size, return_sequences=True),
        
        # Sortie pour chaque timestep
        layers.TimeDistributed(layers.Dense(vocab_size, activation='softmax'))
    ])
    
    return model

# Many-to-One : Classification de séquence
def create_many_to_one_rnn(sequence_length, input_size, hidden_size, num_classes):
    """
    RNN pour classification de séquence (ex: analyse de sentiment).
    """
    model = models.Sequential([
        layers.SimpleRNN(hidden_size, 
                        return_sequences=False,
                        input_shape=(sequence_length, input_size)),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Many-to-Many : Séquence à séquence
def create_many_to_many_rnn(sequence_length, input_size, hidden_size, output_size):
    """
    RNN pour transformation de séquence (ex: NER, POS tagging).
    """
    model = models.Sequential([
        layers.SimpleRNN(hidden_size, 
                        return_sequences=True,  # Retourner toutes les sorties
                        input_shape=(sequence_length, input_size)),
        layers.TimeDistributed(layers.Dense(output_size, activation='softmax'))
    ])
    
    return model

# Créer et afficher les différentes architectures
print("=== One-to-Many (Génération) ===")
model_1tom = create_one_to_many_rnn(512, 256, 20, 10000)
print(f"Input: (batch, 512) → Output: (batch, 20, 10000)\n")

print("=== Many-to-One (Classification) ===")
model_mto1 = create_many_to_one_rnn(50, 300, 128, 5)
print(f"Input: (batch, 50, 300) → Output: (batch, 5)\n")

print("=== Many-to-Many (Tagging) ===")
model_mtom = create_many_to_many_rnn(50, 300, 128, 10)
print(f"Input: (batch, 50, 300) → Output: (batch, 50, 10)")

## 5. Projet Pratique : Analyse de Sentiment

In [ ]:
# Créer un dataset synthétique pour l'analyse de sentiment
def create_sentiment_dataset():
    """
    Crée un dataset simple pour l'analyse de sentiment.
    """
    # Phrases positives
    positive_texts = [
        "Ce film est absolument fantastique, je le recommande vivement",
        "J'ai adoré ce livre, une histoire captivante du début à la fin",
        "Excellent service, personnel très sympathique et professionnel",
        "Produit de qualité exceptionnelle, très satisfait de mon achat",
        "Une expérience merveilleuse, je reviendrai sans hésiter",
        "Superbe performance, les acteurs sont brillants",
        "Un chef-d'œuvre, vraiment impressionnant",
        "Service client au top, problème résolu rapidement",
        "Qualité irréprochable, conforme à la description",
        "Je suis ravi de cette découverte, un vrai coup de cœur"
    ]
    
    # Phrases négatives
    negative_texts = [
        "Film décevant, je me suis ennuyé du début à la fin",
        "Très mauvaise expérience, service déplorable",
        "Produit de mauvaise qualité, ne correspond pas à la description",
        "Je suis très déçu, je ne recommande absolument pas",
        "Perte de temps et d'argent, à éviter",
        "Service client inexistant, aucune réponse à mes questions",
        "Qualité médiocre, très en dessous de mes attentes",
        "Histoire ennuyeuse et prévisible, sans intérêt",
        "Arnaque totale, ne faites pas la même erreur que moi",
        "Catastrophique, je regrette mon achat"
    ]
    
    # Combiner les données
    texts = positive_texts + negative_texts
    labels = [1] * len(positive_texts) + [0] * len(negative_texts)
    
    # Ajouter plus de variété
    for _ in range(5):
        texts.extend(texts)
        labels.extend(labels)
    
    return texts, np.array(labels)

# Créer le dataset
texts, labels = create_sentiment_dataset()
print(f"Nombre d'exemples: {len(texts)}")
print(f"Distribution des labels: {np.bincount(labels)}")
print(f"\nExemples:")
for i in range(3):
    print(f"  {texts[i][:50]}... → {'Positif' if labels[i] else 'Négatif'}")

In [ ]:
# Préparation des données
# Tokenisation et padding
max_words = 1000
max_length = 20

# Créer le tokenizer
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)

# Convertir les textes en séquences
sequences = tokenizer.texts_to_sequences(texts)
print(f"Exemple de séquence: {sequences[0]}")

# Padding des séquences
X = pad_sequences(sequences, maxlen=max_length)
y = labels

# Diviser en train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\nFormes des données:")
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")

In [ ]:
# Créer le modèle RNN pour l'analyse de sentiment
def create_sentiment_rnn(vocab_size, embedding_dim, hidden_dim, max_length):
    """
    Crée un modèle RNN pour l'analyse de sentiment.
    """
    model = models.Sequential([
        # Couche d'embedding
        layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        
        # Couche RNN
        layers.SimpleRNN(hidden_dim, dropout=0.3, recurrent_dropout=0.3),
        
        # Couche de dropout
        layers.Dropout(0.3),
        
        # Couche de sortie
        layers.Dense(1, activation='sigmoid')
    ])
    
    # Compiler le modèle
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Créer le modèle
sentiment_model = create_sentiment_rnn(
    vocab_size=max_words,
    embedding_dim=100,
    hidden_dim=128,
    max_length=max_length
)

sentiment_model.summary()

In [ ]:
# Entraîner le modèle
history = sentiment_model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=10,
    validation_split=0.2,
    verbose=1
)

In [ ]:
# Visualiser l'entraînement
def plot_training_history(history):
    """
    Visualise l'historique d'entraînement.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Loss
    ax1.plot(history.history['loss'], label='Train Loss', linewidth=2)
    ax1.plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    ax1.set_xlabel('Époque')
    ax1.set_ylabel('Loss')
    ax1.set_title('Évolution de la Loss', fontsize=14)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Accuracy
    ax2.plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
    ax2.plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
    ax2.set_xlabel('Époque')
    ax2.set_ylabel('Accuracy')
    ax2.set_title('Évolution de l\'Accuracy', fontsize=14)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_training_history(history)

In [ ]:
# Évaluer le modèle
test_loss, test_accuracy = sentiment_model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

# Prédictions sur de nouvelles phrases
def predict_sentiment(text, model, tokenizer, max_length):
    """
    Prédit le sentiment d'un texte.
    """
    # Prétraitement
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=max_length)
    
    # Prédiction
    prediction = model.predict(padded, verbose=0)[0, 0]
    
    return prediction

# Tester sur de nouvelles phrases
test_phrases = [
    "Ce produit est vraiment excellent, je le recommande",
    "Très déçu de cet achat, qualité médiocre",
    "Pas mal mais peut mieux faire",
    "Une catastrophe totale, à éviter absolument",
    "Superbe expérience, service impeccable"
]

print("\nPrédictions sur de nouvelles phrases:")
print("=" * 60)
for phrase in test_phrases:
    score = predict_sentiment(phrase, sentiment_model, tokenizer, max_length)
    sentiment = "Positif" if score > 0.5 else "Négatif"
    print(f"'{phrase}'")
    print(f"  → {sentiment} (score: {score:.3f})\n")

## 6. Visualisation de l'État Caché

In [ ]:
# Créer un modèle qui retourne les états cachés
def create_rnn_with_states(vocab_size, embedding_dim, hidden_dim, max_length):
    """
    Crée un RNN qui retourne tous les états cachés.
    """
    inputs = layers.Input(shape=(max_length,))
    
    # Embedding
    x = layers.Embedding(vocab_size, embedding_dim)(inputs)
    
    # RNN avec return_sequences=True pour obtenir tous les états
    hidden_states = layers.SimpleRNN(hidden_dim, return_sequences=True)(x)
    
    # Sortie finale (dernier état caché)
    output = layers.Dense(1, activation='sigmoid')(hidden_states[:, -1, :])
    
    # Modèle pour prédiction
    model = models.Model(inputs=inputs, outputs=output)
    
    # Modèle pour obtenir les états cachés
    state_model = models.Model(inputs=inputs, outputs=hidden_states)
    
    return model, state_model

# Créer les modèles
_, state_model = create_rnn_with_states(
    vocab_size=max_words,
    embedding_dim=100,
    hidden_dim=128,
    max_length=max_length
)

# Copier les poids du modèle entraîné
state_model.layers[1].set_weights(sentiment_model.layers[0].get_weights())  # Embedding
state_model.layers[2].set_weights(sentiment_model.layers[1].get_weights())  # RNN

In [ ]:
# Visualiser l'évolution des états cachés
def visualize_hidden_states(text, model, tokenizer, max_length):
    """
    Visualise l'évolution des états cachés pour une phrase.
    """
    # Prétraitement
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=max_length)
    
    # Obtenir les états cachés
    hidden_states = model.predict(padded, verbose=0)[0]
    
    # Obtenir les mots
    words = text.split()
    sequence_words = []
    for idx in sequence[0]:
        for word, word_idx in tokenizer.word_index.items():
            if word_idx == idx:
                sequence_words.append(word)
                break
    
    # Visualisation
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    
    # Heatmap des états cachés
    im = ax1.imshow(hidden_states.T, aspect='auto', cmap='coolwarm')
    ax1.set_xlabel('Position dans la séquence')
    ax1.set_ylabel('Dimension de l\'état caché')
    ax1.set_title(f'États cachés RNN pour: "{text}"', fontsize=14)
    
    # Ajouter les mots
    if len(sequence_words) <= max_length:
        ax1.set_xticks(range(len(sequence_words)))
        ax1.set_xticklabels(sequence_words, rotation=45, ha='right')
    
    plt.colorbar(im, ax=ax1)
    
    # Norme des états cachés
    norms = np.linalg.norm(hidden_states, axis=1)
    ax2.plot(norms, 'o-', linewidth=2, markersize=8)
    ax2.set_xlabel('Position dans la séquence')
    ax2.set_ylabel('Norme de l\'état caché')
    ax2.set_title('Évolution de la norme des états cachés', fontsize=14)
    ax2.grid(True, alpha=0.3)
    
    if len(sequence_words) <= max_length:
        ax2.set_xticks(range(len(sequence_words)))
        ax2.set_xticklabels(sequence_words, rotation=45, ha='right')
    
    plt.tight_layout()
    plt.show()

# Visualiser pour différentes phrases
test_phrases_viz = [
    "Film absolument fantastique",
    "Très mauvais produit décevant"
]

for phrase in test_phrases_viz:
    visualize_hidden_states(phrase, state_model, tokenizer, max_length)

## 7. Comparaison : RNN vs Baseline

In [ ]:
# Créer un modèle baseline (sans récurrence)
def create_baseline_model(vocab_size, embedding_dim, max_length):
    """
    Modèle baseline : moyenne des embeddings + dense.
    """
    model = models.Sequential([
        layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        layers.GlobalAveragePooling1D(),  # Moyenne des embeddings
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Créer et entraîner le modèle baseline
baseline_model = create_baseline_model(max_words, 100, max_length)
print("Modèle Baseline:")
baseline_model.summary()

# Entraîner
baseline_history = baseline_model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=10,
    validation_split=0.2,
    verbose=0
)

# Évaluer
baseline_loss, baseline_acc = baseline_model.evaluate(X_test, y_test, verbose=0)
rnn_loss, rnn_acc = sentiment_model.evaluate(X_test, y_test, verbose=0)

print("\nComparaison des performances:")
print("=" * 40)
print(f"Baseline - Accuracy: {baseline_acc:.4f}, Loss: {baseline_loss:.4f}")
print(f"RNN      - Accuracy: {rnn_acc:.4f}, Loss: {rnn_loss:.4f}")
print(f"\nAmélioration RNN: +{(rnn_acc - baseline_acc)*100:.2f}%")

## 8. Limitations des RNN et Solutions

In [ ]:
# Démonstration du problème du gradient qui disparaît
def demonstrate_vanishing_gradient():
    """
    Montre comment le gradient diminue avec la longueur de la séquence.
    """
    sequence_lengths = [10, 20, 50, 100]
    gradient_norms = []
    
    for seq_len in sequence_lengths:
        # Créer un mini modèle
        model = create_many_to_one_rnn(seq_len, 50, 64, 2)
        
        # Données synthétiques
        x = tf.random.normal((32, seq_len, 50))
        y = tf.random.uniform((32,), maxval=2, dtype=tf.int32)
        
        # Calculer les gradients
        with tf.GradientTape() as tape:
            predictions = model(x, training=True)
            loss = tf.keras.losses.sparse_categorical_crossentropy(y, predictions)
        
        gradients = tape.gradient(loss, model.trainable_variables)
        
        # Calculer la norme moyenne des gradients
        grad_norm = np.mean([tf.norm(g).numpy() for g in gradients if g is not None])
        gradient_norms.append(grad_norm)
    
    # Visualisation
    plt.figure(figsize=(10, 6))
    plt.plot(sequence_lengths, gradient_norms, 'o-', linewidth=2, markersize=10)
    plt.xlabel('Longueur de la séquence')
    plt.ylabel('Norme moyenne des gradients')
    plt.title('Problème du Gradient qui Disparaît dans les RNN', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.yscale('log')  # Échelle logarithmique
    
    # Ajouter des annotations
    for i, (length, norm) in enumerate(zip(sequence_lengths, gradient_norms)):
        plt.annotate(f'{norm:.2e}', 
                    (length, norm), 
                    textcoords="offset points", 
                    xytext=(0,10), 
                    ha='center')
    
    plt.tight_layout()
    plt.show()

demonstrate_vanishing_gradient()

In [ ]:
# Résumé des limitations et solutions
limitations_data = {
    'Limitation': [
        'Gradient qui disparaît',
        'Mémoire à court terme',
        'Pas de parallélisation',
        'Sensibilité à l\'ordre'
    ],
    'Description': [
        'Les gradients deviennent très petits pour les longues séquences',
        'Difficulté à retenir les informations sur de longues distances',
        'Traitement séquentiel obligatoire, lent à entraîner',
        'Performance dégradée si l\'ordre des mots change'
    ],
    'Solution': [
        'LSTM/GRU avec portes',
        'Mécanismes d\'attention',
        'Transformers',
        'Embeddings positionnels'
    ]
}

df_limitations = pd.DataFrame(limitations_data)

# Afficher le tableau
fig, ax = plt.subplots(figsize=(12, 4))
ax.axis('tight')
ax.axis('off')

table = ax.table(cellText=df_limitations.values,
                colLabels=df_limitations.columns,
                cellLoc='left',
                loc='center')

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 2)

# Style du tableau
for i in range(len(df_limitations.columns)):
    table[(0, i)].set_facecolor('#1E90FF')
    table[(0, i)].set_text_props(weight='bold', color='white')

plt.title('Limitations des RNN et Solutions Proposées', fontsize=16, fontweight='bold', pad=20)
plt.show()

## 9. Conclusion et Points Clés

### Ce que nous avons appris :

1. **Architecture RNN** : Comment les RNN traitent les séquences de manière récurrente
2. **Implémentation** : Création de RNN avec TensorFlow/Keras
3. **Types d'architectures** : One-to-One, One-to-Many, Many-to-One, Many-to-Many
4. **Application pratique** : Analyse de sentiment avec RNN
5. **Visualisation** : Comment observer les états cachés évoluent

### Points clés à retenir :

- ✅ Les RNN excellent pour les données séquentielles
- ✅ Ils maintiennent un état caché qui capture le contexte
- ✅ Simples à implémenter avec les frameworks modernes
- ❌ Souffrent du problème du gradient qui disparaît
- ❌ Limités pour les très longues séquences
- ❌ Lents à entraîner (pas de parallélisation)

### Prochaine étape : LSTM et GRU

Les LSTM (Long Short-Term Memory) et GRU (Gated Recurrent Unit) résolvent plusieurs limitations des RNN vanilla en introduisant des mécanismes de portes qui permettent de mieux contrôler le flux d'information.

In [ ]:
# Sauvegarder le modèle pour une utilisation future
sentiment_model.save('rnn_sentiment_model.h5')
print("Modèle sauvegardé sous 'rnn_sentiment_model.h5'")

# Sauvegarder le tokenizer
import pickle
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)
print("Tokenizer sauvegardé sous 'tokenizer.pkl'")